<a href="https://colab.research.google.com/github/HimanshuKakwani/ML-DL/blob/main/stacked_avg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import xgboost as xgb
import lightgbm as lgb
import missingno as mno
from sklearn import linear_model
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/MtM/mine-the-model-celesta-2023/train.csv')
df

,index,rooms,retail_price,address,city,sqft,toilets,image_id
0,5832,2.0,299000,81913 Calle Torbellino,"Indio, CA",1295.0,2.0,5832
1,5584,3.0,299000,26076 Fiesta Place,"Hemet, CA",1566.0,2.0,5584
2,9408,5.0,499990,34450 Fawn Ridge Place,"Yucaipa, CA",3068.0,3.0,9408
3,4644,4.0,849900,26481 Montebello Place,"Mission Viejo, CA",2230.0,2.1,4644
4,5739,3.0,333490,2562 Red Spruce Lane,"San Jacinto, CA",1898.0,2.0,5739
...,...,...,...,...,...,...,...,...
8395,792,4.0,289990,45228 Lorimer Avenue,"Lancaster, CA",1498.0,2.0,792
8396,10671,4.0,895000,265 Old Toll Road,"Lake Arrowhead, CA",2981.0,3.0,10671
8397,11650,3.0,479900,5925 Balmoral Dr,"San Diego, CA",1272.0,1.0,11650
8398,2134,4.0,599000,28137 Stonington Lane,"Saugus, CA",1924.0,3.0,2134


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/MtM/mine-the-model-celesta-2023/test.csv')
df_test


,index,rooms,address,city,sqft,toilets,image_id
0,3076,3,33171 Sea Lion Drive,"Dana Point, CA",1510,2.0,3076
1,3218,4,33 Segovia,"San Clemente, CA",2342,3.0,3218
2,9213,2,1074 TAMARACK Road,"Pinon Hills, CA",2845,3.0,9213
3,10018,4,10989 Coalinga Avenue,"Montclair, CA",2379,3.0,10018
4,11355,4,1128 Custer,"Campo, CA",1836,3.0,11355
...,...,...,...,...,...,...,...
3596,9421,4,14603 Baylor Avenue,"Chino, CA",2092,2.1,9421
3597,219,3,2301 Woodland Drive,"Pine Mountain Club, CA",2000,2.1,219
3598,10333,5,5771 Jasper Street,"Alta Loma, CA",3002,3.1,10333
3599,2814,3,4490 Lincoln Avenue,"Los Angeles, CA",1811,3.1,2814


In [ ]:
#Categorical features
cat_features = ["rooms", "city", "toilets"]

#Continous features
cont_features = ['sqft']

#Feature to be predicted
prediction = "retail_price"

In [ ]:
le = LabelEncoder()
combined = df["city"].append(df_test["city"])
le.fit(combined)

df["city"] = le.transform(df["city"])
df_test["city"] = le.transform(df_test["city"])

In [ ]:
dict(zip(le.classes_, le.transform(le.classes_)))

{'29 Palms, CA': 0,
 'Acton, CA': 1,
 'Adelanto, CA': 2,
 'Agoura Hills, CA': 3,
 'Agua Dulce, CA': 4,
 'Aguanga, CA': 5,
 'Alhambra, CA': 6,
 'Aliso Viejo, CA': 7,
 'Alpine, CA': 8,
 'Alta Loma, CA': 9,
 'Altadena, CA': 10,
 'Anaheim Hills, CA': 11,
 'Anaheim, CA': 12,
 'Angelus Oaks, CA': 13,
 'Anza, CA': 14,
 'Apple Valley, CA': 15,
 'Arcadia, CA': 16,
 'Arleta, CA': 17,
 'Artesia, CA': 18,
 'Arvin, CA': 19,
 'Azusa, CA': 20,
 'Bakersfield, CA': 21,
 'Baldwin Park, CA': 22,
 'Banning, CA': 23,
 'Barstow, CA': 24,
 'Bear Valley Springs, CA': 25,
 'Beaumont, CA': 26,
 'Bell Gardens, CA': 27,
 'Bell, CA': 28,
 'Bellflower, CA': 29,
 'Belltown, CA': 30,
 'Bermuda Dunes, CA': 31,
 'Beverly Hills, CA': 32,
 'Big Bear City, CA': 33,
 'Big Bear Lake, CA': 34,
 'Big Bear, CA': 35,
 'Big River, CA': 36,
 'Bloomington, CA': 37,
 'Blue Jay, CA': 38,
 'Blythe, CA': 39,
 'Bonita, CA': 40,
 'Bonsall, CA': 41,
 'Boron, CA': 42,
 'Borrego Springs, CA': 43,
 'Boulevard, CA': 44,
 'Brawley, CA': 45,
 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8400 entries, 0 to 8399
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         8400 non-null   int64  
 1   rooms         8204 non-null   float64
 2   retail_price  8400 non-null   int64  
 3   address       8400 non-null   object 
 4   city          8400 non-null   int64  
 5   sqft          8253 non-null   float64
 6   toilets       8400 non-null   float64
 7   image_id      8400 non-null   int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 525.1+ KB


In [ ]:
df=df.dropna()

In [ ]:
X = df.drop(labels = prediction, axis = 1)
y = df[prediction]

In [ ]:
# X.drop("address", inplace=True, axis = 1)


In [ ]:
X.drop("image_id", inplace=True,axis = 1)

In [ ]:
X = X[cat_features + cont_features].to_numpy()
y = y.to_numpy()
X.shape, y.shape

((8061, 4), (8061,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
y_train.shape

(6448,)

In [ ]:
X_train.shape

(6448, 4)

In [ ]:
y_test.shape

(1613,)

In [ ]:
X_test.shape

(1613, 4)

In [ ]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train)
    return -cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = kf)

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 97850400538.6677 (2282403448.3762)



In [ ]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 97850353643.8409 (2281809031.7728)



In [ ]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


Kernel Ridge score: 97605693346.1463 (2333335373.3070)



In [ ]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 49731388190.5412 (2377071424.5993)



In [ ]:
# score = rmsle_cv(model_xgb)
# print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
# score = rmsle_cv(model_lgb)
# print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
# class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
#     def __init__(self, models):
#         self.models = models
        
#     # we define clones of the original models to fit the data in
#     def fit(self, X, y):
#         self.models_ = [clone(x) for x in self.models]
        
#         # Train cloned base models
#         for model in self.models_:
#             model.fit(X, y)

#         return self
    
#     #Now we do the predictions for cloned models and average them
#     def predict(self, X):
#         predictions = np.column_stack([
#             model.predict(X) for model in self.models_
#         ])
#         return np.mean(predictions, axis=1) 

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
# averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))

# score = rmsle_cv(averaged_models)
# print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserW

Stacking Averaged models score: 49876251494.2660 (2624732173.6002)


In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
X_test = df_test[cat_features + cont_features].to_numpy()
X_test[0]

array([   3.,   87.,    2., 1510.])

In [ ]:
stacked_averaged_models.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


StackingAveragedModels(base_models=(Pipeline(steps=[('robustscaler',
                                                     RobustScaler()),
                                                    ('elasticnet',
                                                     ElasticNet(alpha=0.0005,
                                                                l1_ratio=0.9,
                                                                random_state=3))]),
                                    GradientBoostingRegressor(learning_rate=0.05,
                                                              loss='huber',
                                                              max_depth=4,
                                                              max_features='sqrt',
                                                              min_samples_leaf=15,
                                                              min_samples_split=10,
                                                              n_estimat

In [ ]:
stacked_averaged_models.fit(X, y)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


StackingAveragedModels(base_models=(Pipeline(steps=[('robustscaler',
                                                     RobustScaler()),
                                                    ('elasticnet',
                                                     ElasticNet(alpha=0.0005,
                                                                l1_ratio=0.9,
                                                                random_state=3))]),
                                    GradientBoostingRegressor(learning_rate=0.05,
                                                              loss='huber',
                                                              max_depth=4,
                                                              max_features='sqrt',
                                                              min_samples_leaf=15,
                                                              min_samples_split=10,
                                                              n_estimat

In [ ]:
df_test["retail_price"] = stacked_averaged_models.predict(X_test)

In [ ]:
df_test

,index,rooms,address,city,sqft,toilets,image_id,retail_price
0,3076,3,33171 Sea Lion Drive,87,1510,2.0,3076,8.341552e+05
1,3218,4,33 Segovia,283,2342,3.0,3218,1.229961e+06
2,9213,2,1074 TAMARACK Road,251,2845,3.0,9213,7.452759e+05
3,10018,4,10989 Coalinga Avenue,203,2379,3.0,10018,7.146640e+05
4,11355,4,1128 Custer,55,1836,3.0,11355,4.225885e+05
...,...,...,...,...,...,...,...,...
3596,9421,4,14603 Baylor Avenue,71,2092,2.1,9421,6.474359e+05
3597,219,3,2301 Woodland Drive,249,2000,2.1,219,4.109034e+05
3598,10333,5,5771 Jasper Street,9,3002,3.1,10333,8.303155e+05
3599,2814,3,4490 Lincoln Avenue,189,1811,3.1,2814,1.011673e+06


In [ ]:
df_sub = df_test.drop(columns=["image_id", "address"] + cont_features + cat_features, axis = 1)
df_sub

,index,retail_price
0,3076,8.341552e+05
1,3218,1.229961e+06
2,9213,7.452759e+05
3,10018,7.146640e+05
4,11355,4.225885e+05
...,...,...
3596,9421,6.474359e+05
3597,219,4.109034e+05
3598,10333,8.303155e+05
3599,2814,1.011673e+06


In [ ]:
df_sub.set_index('index', inplace = True)
df_sub

,retail_price
index,
3076,8.341552e+05
3218,1.229961e+06
9213,7.452759e+05
10018,7.146640e+05
11355,4.225885e+05
...,...
9421,6.474359e+05
219,4.109034e+05
10333,8.303155e+05


In [ ]:
df_sub.to_csv('AVG2.csv')